## Set 5
## 3. Word2Vec \*\*Principles**

#### Preparation

In [ ]:
# download the helper function
!wget -O P3CHelpers.py https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/P3CHelpers.py

--2022-02-16 05:16:59--  https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/P3CHelpers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4939 (4.8K) [text/plain]
Saving to: ‘P3CHelpers.py’

P3CHelpers.py       100%[===================>]   4.82K  --.-KB/s    in 0s      

2022-02-16 05:16:59 (46.8 MB/s) - ‘P3CHelpers.py’ saved [4939/4939]



In [ ]:
# download the dataset
!wget -O dr_seuss.txt https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/data/dr_seuss.txt

--2022-02-16 05:16:59--  https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/data/dr_seuss.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8810 (8.6K) [text/plain]
Saving to: ‘dr_seuss.txt’

dr_seuss.txt        100%[===================>]   8.60K  --.-KB/s    in 0s      

2022-02-16 05:17:00 (60.6 MB/s) - ‘dr_seuss.txt’ saved [8810/8810]



In [ ]:
import numpy as np
from P3CHelpers import *
import torch
import torch.nn as nn
import torch.optim as optim

#### Problem D: 
Fill in the generate_traindata and find_most_similar_pairs functions.

In [ ]:
def get_word_repr(word_to_index, word):
    """
    Returns one-hot-encoded feature representation of the specified word given
    a dictionary mapping words to their one-hot-encoded index.

    Arguments:
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        word:          Word whose feature representation we wish to compute.

    Returns:
        feature_representation:     Feature representation of the passed-in word.
    """
    unique_words = word_to_index.keys()
    # Return a vector that's zero everywhere besides the index corresponding to <word>
    feature_representation = np.zeros(len(unique_words))
    feature_representation[word_to_index[word]] = 1
    return feature_representation    

def generate_traindata(word_list, word_to_index, window_size=4):
    """
    Generates training data for Skipgram model.

    Arguments:
        word_list:     Sequential list of words (strings).
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        window_size:   Size of Skipgram window. Defaults to 2 
                       (use the default value when running your code).

    Returns:
        (trainX, trainY):     A pair of matrices (trainX, trainY) containing training 
                              points (one-hot-encoded vectors) and their corresponding output_word
                              (also one-hot-encoded vectors)

    """
    trainX = []
    trainY = []
    
    ##############################################################
    # TODO: Implement this function, populating trainX and trainY
    ##############################################################
    for i,word in enumerate(word_list):
        x = get_word_repr(word_to_index, word)
        for j in range(1, window_size + 1):
            if i - j >= 0:
                y1 = get_word_repr(word_to_index, word_list[i-j])
                trainX.append(x)
                trainY.append(y1)
        for j in range(1, window_size + 1):
            if i + j < len(word_list):
                y2 = get_word_repr(word_to_index, word_list[i+j])
                trainX.append(x)
                trainY.append(y2)
    
    return np.array(trainX), np.array(trainY)

In [ ]:
def find_most_similar_pairs(filename, num_latent_factors):
    """
    Find the most similar pairs from the word embeddings computed from
    a body of text
    
    Arguments:
        filename:           Text file to read and train embeddings from
        num_latent_factors: The number of latent factors / the size of the embedding
    """
    # Load in a list of words from the specified file; remove non-alphanumeric characters
    # and make all chars lowercase.
    sample_text = load_word_list(filename)

    # Create word dictionary
    word_to_index = generate_onehot_dict(sample_text)
    print("Textfile contains %s unique words"%len(word_to_index))
    # Create training data
    trainX, trainY = generate_traindata(sample_text, word_to_index)

    ##############################################################
    # TODO: 1) Create and train model in Pytorch.      
    ##############################################################
    
    # vocab_size = number of unique words in our text file. Will be useful 
    # when adding layers to your neural network
    vocab_size = len(word_to_index)
    model = nn.Sequential(
        nn.Linear(in_features=vocab_size, out_features=num_latent_factors),
        nn.Linear(in_features=num_latent_factors, out_features=vocab_size),
        nn.Softmax(dim=-1)
    )

    traindata = torch.from_numpy(np.array([trainX,trainY]).astype(np.float32))
    train_loader = torch.utils.data.DataLoader(traindata, batch_size=256, shuffle=True)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.025)
    loss_fn = nn.CrossEntropyLoss()
    # Some layers, such as Dropout, behave differently during training
    model.train()

    for epoch in range(15):
        for batch_idx, (data, target) in enumerate(train_loader):
            # Erase accumulated gradients
            optimizer.zero_grad()

            if epoch==0:
                print(data.size())
                print(target.size())

            # Forward pass
            output = model(data)

            # Calculate loss
            loss = loss_fn(output, target)

            # Backward pass
            loss.backward()
            
            # Weight update
            optimizer.step()

        # Track loss each epoch
        print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))

    ##############################################################
    # TODO: 2) Extract weights for hidden layer
    ##############################################################
    for param in model.parameters():
        param.requires_grad = False

    # set weights variable below
    
    print(list(model.parameters())[0].size())
    print(list(model.parameters())[1].size())
    print(list(model.parameters())[2].size())
    print(list(model.parameters())[3].size())
    weights = list(model.parameters())[2]
    
    # Find and print most similar pairs
    similar_pairs = most_similar_pairs(weights, word_to_index)
    for pair in similar_pairs[:30]:
        print(pair)

### Problem E-H:
Run your model on drseuss.txt and answer questions from E through H.

In [ ]:
find_most_similar_pairs('dr_seuss.txt', 10)

Textfile contains 308 unique words
torch.Size([16548, 308])
torch.Size([16548, 308])
Train Epoch: 1  Loss: 5.7301
Train Epoch: 2  Loss: 5.7300
Train Epoch: 3  Loss: 5.7298
Train Epoch: 4  Loss: 5.7295
Train Epoch: 5  Loss: 5.7292
Train Epoch: 6  Loss: 5.7288
Train Epoch: 7  Loss: 5.7282
Train Epoch: 8  Loss: 5.7274
Train Epoch: 9  Loss: 5.7264
Train Epoch: 10  Loss: 5.7252
Train Epoch: 11  Loss: 5.7236
Train Epoch: 12  Loss: 5.7216
Train Epoch: 13  Loss: 5.7193
Train Epoch: 14  Loss: 5.7166
Train Epoch: 15  Loss: 5.7135
torch.Size([10, 308])
torch.Size([10])
torch.Size([308, 10])
torch.Size([308])
Pair(more, ish), Similarity: 0.99268425
Pair(ish, more), Similarity: 0.99268425
Pair(ear, bird), Similarity: 0.9898995
Pair(bird, ear), Similarity: 0.9898995
Pair(something, zeep), Similarity: 0.9848032
Pair(zeep, something), Similarity: 0.9848032
Pair(long, head), Similarity: 0.98151547
Pair(head, long), Similarity: 0.98151547
Pair(dog, about), Similarity: 0.9786716
Pair(about, dog), Similar